## Saúde Bucal - material suplementar


### Procedimentos metodológicos
---
A construção do modelo de planejamento da força de trabalho (PDFT) para saúde bucal baseado em necessidades foi elaborado a partir das orientações de Asamani et al. (2021) que sugerem algumas etapas, que serão descritas com maior detalhamento a seguir:

1. Definição de escopo da metodologia em termos de cobertura jurisdicional, objetivos e horizonte temporal;
2. Análise da necessidade de saúde bucal com base em aspectos da demografia, epidemiologia e serviços destinados da população a local;
3. Tradução do número de serviços em número de profissionais necessários;
4. Exploração das implicações em termos de recursos, como a oferta atual de profissionais e custos;
5. Análise de sensibilidade para simulação de parâmetros de incerteza;
6. Validação de modelo por meio de consulta a stakeholders.

### Preparando ambiente do R
---

Carregando os pacotes necessários para o Python.

In [30]:
#instalando os pacotes que ainda não foram instalados

!pip install pyodbc
!pip install geopandas
!pip install rtree
!pip install pygeos
!pip install mapclassify
!pip install geobr
!pip install plotnine
!pip install dash
!pip install cartopy
!pip install geojson


In [31]:
# Modelagem
import pandas as pd
import numpy as np

#execução dremio
import pyodbc

# Data View
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib.colors import Normalize
from matplotlib import cm
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
import plotnine
import dash
import dash.dash_table
import adjustText
import matplotlib.gridspec

#Mapas
import geopandas as gpd
import rtree
import pygeos
import mapclassify
import geobr
from matplotlib.ticker import FuncFormatter
import cartopy

import geojson

### 1. Definição de escopo
---
A metodologia tem como objetivo estimar a força de trabalho atual e necessária de cirurgiões dentistas e técnicos e auxilares de saúde bucal para atuação em municípios brasileiros, a nível de atenção primária, em procedimentos de natureza individual, bem como três procedimentos de natureza especializada: endodontia, prótese e periodontia.


### 2. Análise das necessidades de saúde bucal
---
A análise da necessidade de saúde bucal (NSB) foi construída a partir de três elementos: distribuição da população por faixa etária (P), prevalência de condições de saúde bucal (H) e procedimentos per capita (S).

![image.png](imagens/equacao1_sb.png)

Onde:

* NSB = Necessidade de saúde bucal é o número de serviços do tipo t destinados ao atendimento de população de faixa etária i e localidade l;
* P = População por faixa-etária i de uma localidade l;
* H = Prevalência de condições de saúde bucal que levam à necessidade de procedimentos do tipo t por faixa-etária i e dentro de uma localidade l;
* S = Procedimentos per capita do tipo t, para cada faixa-etária i e dentro de uma localidade l.

### 2.1. Levantando a faixa etária (P)

A distribuição por faixa etária foi construída com base em dados do último censo (2022) e estão disponíveis na página do IBGE.

In [80]:
teste = pd.read_excel("populacao_municipio_censo_completa.xlsx")
teste

      cod_municipiodv        municipio  ibge_sb  0 a 4 anos  5 a 9 anos  \
0             5300108         Brasília  5300108      166848      184611   
1             5208707          Goiânia  5208707       83676       89501   
2             5200050  Abadia de Goiás  5200000        1590        1619   
3             5200100        Abadiânia  5200000        1171        1228   
4             5200134          Acreúna  5200000        1384        1497   
...               ...              ...      ...         ...         ...   
5764          1101609        Theobroma  1300000         521         533   
5765          1101708            Urupá  1300000         741         713   
5766          1101757    Vale do Anari  1300000         516         597   
5767          1101807  Vale do Paraíso  1300000         381         413   
5768          1100304          Vilhena  1300000        7158        6948   

      10 a 14 anos  15 a 19 anos  20 a 24 anos  25 a 29 anos  30 a 34 anos  \
0           182774   